In [ ]:
# Associate words with archetypes/character traits as intermediate layer
# and with influencer as the "last" layer

# Dependencies
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import os
import toml
import re
import itertools
from text_cleaner import *
import operator
from collections import Counter
import pickle

def extract_hashtags(post_text):
    HASH_RE = re.compile(r"\#\w+")
    out_list = re.findall(HASH_RE, post_text)
    return out_list

In [2]:
# Load the .csv with archetypes
arch_df = pd.read_csv('archetypes_pl.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [3]:
# Table preprocessing - replace all NaN with 2 (Unrelated/Don't know class), replace 0-5 values with the ones in range -1.0 - 1.0
arch_df = arch_df.fillna(2)

# Remove duplicated annotations, to exclude conflicting entries
arch_df = arch_df[~arch_df.index.duplicated(keep='first')]

# Print the head of the dataset after modification
arch_df.head()

,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [4]:
# Check if a user has a non-empty directory in the dataset, otherwise delete the user from the list
available_arch_df = copy.deepcopy(arch_df)
posts = []

BASE_DIR = "instagram_cleared"

# Iterate over whole DataFrame
for i, row in tqdm(arch_df.iterrows()):
    profile_posts = []
    profile_hashtags = []
    
    # Get all posts per profile
    profile_path = os.path.join(BASE_DIR, i)
    for file in os.listdir(profile_path):
        if not file.endswith(".toml"):
            with open(os.path.join(profile_path, file), "r") as post_f:
                read_text = post_f.read()
                profile_posts.append(remove_stopwords(clean_up_text(read_text)))
                profile_hashtags.append(extract_hashtags(read_text))

    # Merge lists - a single list for a single influencer
    profile_hashtags = list(itertools.chain.from_iterable(profile_hashtags))
    posts.append(list(itertools.chain.from_iterable([profile_posts, [profile_hashtags]])))

508it [00:12, 41.19it/s]


In [5]:
# Map usernames to indices
users = list(available_arch_df.index.values)
user_indices = {k: users.index(k) for k in users}

In [6]:
# Get the word count and create a dataframe, where columns are archetypes/traits, and rows are single words
# Initialize a word DataFrame
word_df = pd.DataFrame()

def merge_dicts(dict_a, dict_b) -> dict:
    out_dict = dict_a
    for k, v in dict_b.items():
        if k in out_dict.keys():
            out_dict[k] += v
        else:
            out_dict[k] = v
    return out_dict

# Iterate over all of the traits/archetypes
for trait in tqdm(trait_list):
    # Select influencers which have the given archetype annotated
    subset_df = arch_df[trait]
    subset_indices = [user_indices[idx] for idx in subset_df.index.values]
    trait_weights = subset_df.tolist()
    
    # Get all posts for the list of influencers
    f = operator.itemgetter(*subset_indices)
    sublist = list(f(posts))
    
    # Counter to calculate each word occurrences
    trait_total = 0
    out_dict = {}
    for i, post_set in enumerate(sublist):
        trait_ctr = Counter(itertools.chain.from_iterable(post_set))
        trait_total += sum(trait_ctr.values())
        for key in trait_ctr:
            trait_ctr[key] *= trait_weights[i]
        out_dict = merge_dicts(out_dict, trait_ctr)
    out_dict = {k: float(v / trait_total) for k, v in out_dict.items()}
    trait_ctr = {trait: out_dict}
    
    # Append the new dataframe
    tmp_df = pd.DataFrame.from_dict(trait_ctr, orient="index")
    word_df = word_df.append(tmp_df)

100%|██████████| 37/37 [04:59<00:00,  8.10s/it]


In [6]:
# AGDS - discrete approach, classification-like
# Try to associate word with trait association class, not the trait itself

def merge_dicts(dict_a, dict_b) -> dict:
    out_dict = dict_a
    for k, v in dict_b.items():
        if k in out_dict.keys():
            out_dict[k] += v
        else:
            out_dict[k] = v
    return out_dict

# Iterate over all of the traits/archetypes
word_set = set()
trait_df_list = []
for trait in tqdm(trait_list):
    # Select influencers which have the given archetype annotated
    subset_df = arch_df[trait]
    trait_subframe = pd.DataFrame()
    for trait_class in range(5):
        class_df = subset_df.loc[subset_df == trait_class]
        subset_indices = [user_indices[idx] for idx in class_df.index.values]

        # Get all posts for the list of influencers
        f = operator.itemgetter(*subset_indices)
        sublist = list(f(posts))

        # Counter to calculate each word occurrences
        trait_total = 0
        out_dict = {}
        for i, post_set in enumerate(sublist):
            trait_ctr = Counter(itertools.chain.from_iterable(post_set))
            trait_total += sum(trait_ctr.values())
            out_dict = merge_dicts(out_dict, trait_ctr)
        out_dict = {k: float(v / trait_total) for k, v in out_dict.items()}
        word_set.update(out_dict.keys())
        trait_ctr = {trait_class: out_dict}
        trait_tmp_df = pd.DataFrame.from_dict(trait_ctr, orient="index")
        trait_subframe = trait_subframe.append(trait_tmp_df)

    # Append the new dataframe
    #word_df = word_df.append(trait_subframe)
    trait_df_list.append(trait_subframe)
    
softmax_word_df = pd.concat(trait_df_list, keys=trait_list)

100%|██████████| 37/37 [12:50<00:00, 20.84s/it]


In [7]:
# Check the calculation results
softmax_word_df

zainspirowany  wczorajszy    wywiad  odnośnie   relacja  \
innocent    0       0.000021    0.000085  0.000021  0.000043  0.000298   
            1            NaN    0.000122  0.000027  0.000014  0.000595   
            2       0.000003    0.000134  0.000073  0.000043  0.000379   
            3       0.000006    0.000178  0.000024  0.000036  0.000525   
            4            NaN    0.000130  0.000012  0.000059  0.000272   
...                      ...         ...       ...       ...       ...   
allocentric 0       0.000017    0.000086  0.000069  0.000052  0.000449   
            1       0.000005    0.000131  0.000010  0.000039  0.000535   
            2            NaN    0.000152  0.000076  0.000028  0.000290   
            3       0.000005    0.000165  0.000033  0.000046  0.000474   
            4       0.000003    0.000169  0.000039  0.000026  0.000474   

                 chcieć  przekazać     okres  kwarantanny   świetny  ...  \
innocent    0  0.002450   0.000064  0.000362     0.000128  0.000682  ...   
            1  0.002705   0.000041  0.000149     0.000041  0.000162  ...   
            2  0.002418   0.000051  0.000247     0.000035  0.000349  ...   
            3  0.002308   0.000040  0.000155     0.000039  0.000304  ...   
            4  0.002154   0.000012  0.000189     0.000047  0.000154  ...   
...                 ...        ...       ...          ...       ...  ...   
allocentric 0  0.002950   0.000104  0.000190     0.000069  0.000397  ...   
            1  0.002345   0.000024  0.000092     0.000054  0.000238  ...   
            2  0.001982   0.000021  0.000380     0.000014  0.000318  ...   
            3  0.002295   0.000022  0.000186     0.000038  0.000342  ...   
            4  0.002548   0.000091  0.000179     0.000049  0.000289  ...   

               #ciemneklimaty  #dziewczynkab  #dziewczynaks  \
innocent    0             NaN            NaN            NaN   
            1             NaN            NaN            NaN   
            2             NaN            NaN            NaN   
            3             NaN            NaN            NaN   
            4        0.000012       0.000012       0.000012   
...                       ...            ...            ...   
allocentric 0             NaN            NaN            NaN   
            1             NaN            NaN            NaN   
            2             NaN            NaN            NaN   
            3             NaN            NaN            NaN   
            4        0.000003       0.000003       0.000003   

               #spacerzleśnymechem_kobiecość  #samamama  #kobietamama  \
innocent    0                            NaN        NaN           NaN   
            1                            NaN        NaN           NaN   
            2                            NaN        NaN           NaN   
            3                            NaN        NaN           NaN   
            4                       0.000012   0.000012      0.000012   
...                                      ...        ...           ...   
allocentric 0                            NaN        NaN           NaN   
            1                            NaN        NaN           NaN   
            2                            NaN        NaN           NaN   
            3                            NaN        NaN           NaN   
            4                       0.000003   0.000003      0.000003   

               #DZIENMATKI  #tematnadziś  #wieczory  #jellycat  
innocent    0          NaN           NaN        NaN        NaN  
            1          NaN           NaN        NaN        NaN  
            2          NaN           NaN        NaN        NaN  
            3          NaN           NaN        NaN        NaN  
            4     0.000012      0.000012   0.000012   0.000012  
...                    ...           ...        ...        ...  
allocentric 0          NaN           NaN        NaN        NaN  
            1          NaN           NaN        NaN        

In [8]:
# Fill NaN values with 0
softmax_word_df = softmax_word_df.fillna(0)

import pickle

with open("softmax_lemma_influencer_index_map.pickle", "wb") as f:
    pickle.dump(user_indices, f)
    
softmax_word_df.to_pickle("softmax_lemma_word_trait_array.pickle")

In [7]:
# Fill NaN values with 0
word_df = word_df.fillna(0)

In [8]:
# Save the result to a pickle
import pickle

with open("v5_influencer_index_map.pickle", "wb") as f:
    pickle.dump(user_indices, f)
    
word_df.to_pickle("v5_word_trait_array.pickle")

In [9]:
# Create word map and save it to a pickle (this file must be loaded in pair with word_trait_array)
word_map = word_df.columns.tolist()

with open("new_word_map.pickle", "wb") as f:
    pickle.dump(word_map, f)

In [12]:
def get_trait_dot_product(post_text: str, word_map: list, word_dataframe: pd.DataFrame) -> list:
    # Filter out the text
    filtered_post = remove_stopwords(clean_up_text(post_text))
    filtered_post += extract_hashtags(post_text)
    
    # Create a vector for dot product vector
    post_vector = [0] * len(word_map)
    
    # Calculate word occurrences
    word_ctr = Counter(filtered_post)
    
    for word, freq in word_ctr.items():
        if word in word_map:
            post_vector[word_map.index(word)] = freq
    
    # Calculate dot product for a given text
    word_dot = word_dataframe.dot(post_vector)
    
    return word_dot

# Replace NaN with 0 in word_frequency_table
word_df = word_df.fillna(0)

# Method for calculating the dot product of trait <-> influencer relation
def get_influencer_dot_product(trait_output: list, influencer_dataframe: pd.DataFrame) -> pd.DataFrame:
    return influencer_dataframe.dot(trait_output)

# Method for calculating the similarity
def calculate_similarity(post_text: str, 
                         word_map: list, 
                         word_dataframe: pd.DataFrame,
                         influencer_dataframe: pd.DataFrame) -> pd.DataFrame:
    # Calculate word-trait dot product
    post_result = get_trait_dot_product(post_text, word_map, word_dataframe)
    
    # Calculate trate-influencer dot-product
    inf_dot_product = get_influencer_dot_product(post_result, influencer_dataframe)
    
    # Get the sum of influencer traits
    influencer_sum = influencer_dataframe.sum(axis=1)
    
    # Divide the dot product by the sum calculated above
    inf_dot_product = inf_dot_product.divide(influencer_sum)
    
    return inf_dot_product

# Trait accuracy - round the results
def natural_round(x: float) -> int:
    out = int(x // 1)
    return out + 1 if (x - out) >= 0.5 else out

def accuracy_per_trait(input_vector: pd.Series, annotated_vector: pd.Series) -> np.array:
    out_array = np.array([0] * 37, dtype=np.int)
    for i in range(len(out_array)):
        if natural_round(input_vector[i]) == annotated_vector[i]:
            out_array[i] = 1
    return out_array

In [14]:
pbar = tqdm(arch_df.iterrows())

# Out accuracy vector
total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    user_text = list(itertools.chain.from_iterable(posts[users.index(idx)]))
    user_text = " ".join(user_text)
    sim_output = get_trait_dot_product(user_text, word_map, word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    total_accuracy += user_accuracy
    pbar.set_description(f"Average accuracy: {round(np.mean(np.divide(total_accuracy, users.index(idx)+1))*100, 2)}")

Average accuracy: 15.48: : 508it [24:26,  2.89s/it]


# Softmax version

In [15]:
from scipy.special import softmax

def get_trait_dot_product(post_text: str, word_map: list, word_dataframe: pd.DataFrame) -> list:
    # Filter out the text
    filtered_post = remove_stopwords(clean_up_text(post_text))
    filtered_post += extract_hashtags(post_text)
    
    # Create a vector for dot product vector
    post_vector = [0] * len(word_map)
    
    # Calculate word occurrences
    word_ctr = Counter(filtered_post)
    
    for word, freq in word_ctr.items():
        if word in word_map:
            post_vector[word_map.index(word)] = freq
    
    # Calculate dot product for a given text
    word_dot = word_dataframe.dot(post_vector)
    
    out_vec = pd.Series()
    for trait in trait_list:
        out_vec = out_vec.append(pd.Series([np.argmax(softmax(word_dot.loc[trait]))], index=[trait]))
    
    return out_vec

# Trait accuracy - round the results
def natural_round(x: float) -> int:
    out = int(x // 1)
    return out + 1 if (x - out) >= 0.5 else out

def accuracy_per_trait(input_vector: pd.Series, annotated_vector: pd.Series) -> np.array:
    out_array = np.array([0] * 37, dtype=np.int)
    for i in range(len(out_array)):
        if input_vector[i] == annotated_vector[i]:
            out_array[i] = 1
    return out_array

In [16]:
pbar = tqdm(arch_df.iterrows())
accuracy = 0

# Out accuracy vector
total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    user_text = list(itertools.chain.from_iterable(posts[users.index(idx)]))
    user_text = " ".join(user_text)
    sim_output = get_trait_dot_product(user_text, word_map, softmax_word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    total_accuracy += user_accuracy
    pbar.set_description(f"Average accuracy: {round(np.mean(np.divide(total_accuracy, users.index(idx)+1))*100, 2)}")

0it [00:00, ?it/s]


NameError: name 'softmax_word_df' is not defined

In [24]:
# Test dataset
softmax_word_map = softmax_word_df.columns.tolist()

# Load the .csv with archetypes
arch_df = pd.read_csv('test_archetypes_pl.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

# Table preprocessing - replace all NaN with 2 (Unrelated/Don't know class), replace 0-5 values with the ones in range -1.0 - 1.0
arch_df = arch_df.fillna(2)

# Remove duplicated annotations, to exclude conflicting entries
arch_df = arch_df[~arch_df.index.duplicated(keep='first')]

# Print the head of the dataset after modification
arch_df.head()

# Check if a user has a non-empty directory in the dataset, otherwise delete the user from the list
available_arch_df = copy.deepcopy(arch_df)
posts = []

BASE_DIR = "instagram_cleared"

# Iterate over whole DataFrame
for i, row in tqdm(arch_df.iterrows()):
    profile_posts = []
    profile_hashtags = []
    
    # Get all posts per profile
    profile_path = os.path.join(BASE_DIR, i)
    for file in os.listdir(profile_path):
        if not file.endswith(".toml"):
            with open(os.path.join(profile_path, file), "r") as post_f:
                read_text = post_f.read()
                profile_posts.append(remove_stopwords(clean_up_text(read_text)))
                profile_hashtags.append(extract_hashtags(read_text))

    # Merge lists - a single list for a single influencer
    profile_hashtags = list(itertools.chain.from_iterable(profile_hashtags))
    posts.append(list(itertools.chain.from_iterable([profile_posts, [profile_hashtags]])))
    
# Map usernames to indices
users = list(arch_df.index.values)
user_indices = {k: users.index(k) for k in users}



0it [00:00, ?it/s]

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


177it [01:25,  2.07it/s]


In [28]:
pbar = tqdm(arch_df.iterrows())

# Out accuracy vector
test_total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    profile_path = os.path.join(BASE_DIR, idx)
    user_text = ""
    for file in os.listdir(profile_path):
        if not file.endswith(".toml"):
            with open(os.path.join(profile_path, file), "r") as post_f:
                read_text = post_f.read()
                user_text += read_text
    sim_output = get_trait_dot_product(user_text, softmax_word_map, softmax_word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    test_total_accuracy += user_accuracy
    pbar.set_description(f"Average test dataset accuracy: {round(np.mean(np.divide(test_total_accuracy, users.index(idx)+1))*100, 2)}")
    
# Show total accuracy
scaled_test_accuracy = np.divide(test_total_accuracy, len(arch_df))
avg_test_accuracy = np.mean(scaled_test_accuracy)

print("--- ACCURACY ON TESTING DATASET ---")

print(f"Average test dataset accuracy: {round(avg_test_accuracy*100, 2)}%")
print("Accuracy per trait:")
for i in range(len(trait_list)):
    print(f"{trait_list[i]}: {round(scaled_test_accuracy[i] * 100, 2)}%")

0it [00:00, ?it/s]<ipython-input-20-4ffef00e153a>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out_vec = pd.Series()
Average test dataset accuracy: 21.62: : 4it [00:14,  3.51s/it]


KeyboardInterrupt: 